# Module 4: Hourly Updates

## Purpose
This module runs hourly to monitor and adjust prices based on:
1. **UTH (Up-Till-Hour) Performance**: Cumulative qty/retailers from start of day until current hour
2. **Last Hour Performance**: Qty/retailers for the most recent hour only

## Schedule
- Runs hourly from 12 PM to 12 AM (midnight), **except** Module 3 hours (12 PM, 3 PM, 6 PM, 9 PM)
- Also runs once at 3 AM
- Active hours: 1 PM, 2 PM, 4 PM, 5 PM, 7 PM, 8 PM, 10 PM, 11 PM, 12 AM, 3 AM

## Data Flow
```
data_extraction.ipynb → Snowflake (Pricing_data_extraction)
                              ↓
                        Module 4 (this module)
                           ├── Load p80_qty, p70_retailers, std columns
                           ├── Fetch fresh: cart rules, stocks, WAC
                           ├── Calculate new_wac (from today's PRS)
                           ├── Query UTH performance (today)
                           ├── Query Last Hour performance (today)
                           ├── Query historical hour contributions
                           ├── Calculate targets & statuses (±3 std)
                           └── Generate actions (TBD)
```

## New WAC Calculation
To avoid database lag, we calculate a fresh WAC from today's purchase receipts:
- `new_wac1` = ((stocks + reflected_qty) × wac1 + unreflected_qty × item_price) / total_qty
- `new_wac_p` = wac_p × (1 + wac1_change)
- `new_wac` = new_wac_p (with fallback to current_wac)

## Status Logic (±3 Standard Deviations)
SKU status is determined by comparing actual performance to target ± 3 std:
- **Growing**: actual > target + 3×std
- **On Track**: target - 3×std ≤ actual ≤ target + 3×std
- **Dropping**: actual < target - 3×std (minimum threshold = 1)

Standard deviation columns used:
- Qty: `std_daily_240d`
- Retailers: `std_daily_retailers_240d`

## Status Outputs
- `uth_qty_status`: growing / dropping / on_track
- `uth_rets_status`: growing / dropping / on_track
- `last_hour_qty_status`: growing / dropping / on_track
- `last_hour_rets_status`: growing / dropping / on_track


In [1]:
# =============================================================================
# IMPORTS AND SETUP
# =============================================================================
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import sys
sys.path.append('..')
import setup_environment_2
# Import queries module for Snowflake access
%run queries_module.ipynb

# Cairo timezone
CAIRO_TZ = pytz.timezone('Africa/Cairo')
CAIRO_NOW = datetime.now(CAIRO_TZ)
CURRENT_HOUR = CAIRO_NOW.hour

print(f"Module 4: Hourly Updates")
print(f"Current Cairo Time: {CAIRO_NOW.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Current Hour: {CURRENT_HOUR}")


/home/ec2-user/anaconda3/envs/python3/lib/python3.12/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (20.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


/home/ec2-user/.Renviron
/home/ec2-user/service_account_key.json
Queries Module | Timezone: America/Los_Angeles
✅ UTH and Last Hour functions defined

QUERIES MODULE READY

Live Data Functions:
  • get_current_stocks()
  • get_packing_units()
  • get_current_prices()
  • get_current_wac()
  • get_current_cart_rules()

UTH Performance Functions:
  • get_uth_performance()         - UTH qty/retailers (Snowflake)
  • get_hourly_distribution()     - Historical hour contributions (Snowflake)
  • get_last_hour_performance()   - Last hour qty/retailers (DWH)

Note: Market prices use MODULE_1_INPUT data
Retailer Selection Queries defined ✓
  - get_churned_dropped_retailers()
  - get_category_not_product_retailers()
  - get_out_of_cycle_retailers()
  - get_view_no_orders_retailers()
  - get_excluded_retailers()
  - get_retailers_with_quantity_discount()
  - get_retailer_main_warehouse()
Module 4: Hourly Updates
Current Cairo Time: 2026-01-27 13:20:14
Current Hour: 13


In [2]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# Input/Output configuration
# Data is now loaded from Snowflake instead of Excel
INPUT_TABLE = 'MATERIALIZED_VIEWS.Pricing_data_extraction'
OUTPUT_FILE = f'module_4_output_{CAIRO_NOW.strftime("%Y%m%d_%H%M")}.xlsx'

# Status thresholds (±3 std from target)
# - Growing: actual > target + 3*std
# - On Track: target - 3*std <= actual <= target + 3*std
# - Dropping: actual < target - 3*std (minimum threshold = 1)
STD_THRESHOLD = 3  # Number of standard deviations
MIN_DROPPING_THRESHOLD = 1  # Minimum threshold for dropping status

# Module 3 hours (skip these)
MODULE_3_HOURS = [12, 15, 18, 21]  # 12 PM, 3 PM, 6 PM, 9 PM

print(f"Input: {INPUT_TABLE} (today's data)")
print(f"Output: {OUTPUT_FILE}")
print(f"Status Thresholds: ±{STD_THRESHOLD} std (Dropping minimum = {MIN_DROPPING_THRESHOLD})")


Input: MATERIALIZED_VIEWS.Pricing_data_extraction (today's data)
Output: module_4_output_20260127_1320.xlsx
Status Thresholds: ±3 std (Dropping minimum = 1)


In [3]:
# =============================================================================
# LOAD DATA FROM SNOWFLAKE (Instead of Excel file)
# =============================================================================
print("Loading data from Snowflake...")

# Query to get today's data from Pricing_data_extraction
LOAD_QUERY = f"""
SELECT * FROM {INPUT_TABLE}
WHERE created_at = '{datetime.now(CAIRO_TZ).date()}'
"""

df = query_snowflake(LOAD_QUERY)
print(f"Loaded {len(df)} records from Snowflake")

# Ensure required columns exist with proper types
df['p80_daily_240d'] = pd.to_numeric(df.get('p80_daily_240d', 0), errors='coerce').fillna(0)
df['p70_daily_retailers_240d'] = pd.to_numeric(df.get('p70_daily_retailers_240d', 1), errors='coerce').fillna(1)
df['std_daily_240d'] = pd.to_numeric(df.get('std_daily_240d', 0), errors='coerce').fillna(0)
df['std_daily_retailers_240d'] = pd.to_numeric(df.get('std_daily_retailers_240d', 0), errors='coerce').fillna(0)
df['warehouse_id'] = df['warehouse_id'].astype(int)
df['product_id'] = df['product_id'].astype(int)
df['cohort_id'] = df['cohort_id'].astype(int) if 'cohort_id' in df.columns else None

# Get category for hourly distribution merge
if 'cat' not in df.columns and 'category' in df.columns:
    df['cat'] = df['category']

print(f"\nP80 Qty Stats: min={df['p80_daily_240d'].min():.1f}, max={df['p80_daily_240d'].max():.1f}, mean={df['p80_daily_240d'].mean():.1f}")
print(f"P70 Retailers Stats: min={df['p70_daily_retailers_240d'].min():.1f}, max={df['p70_daily_retailers_240d'].max():.1f}, mean={df['p70_daily_retailers_240d'].mean():.1f}")
print(f"Std Qty Stats: min={df['std_daily_240d'].min():.1f}, max={df['std_daily_240d'].max():.1f}, mean={df['std_daily_240d'].mean():.1f}")
print(f"Std Retailers Stats: min={df['std_daily_retailers_240d'].min():.1f}, max={df['std_daily_retailers_240d'].max():.1f}, mean={df['std_daily_retailers_240d'].mean():.1f}")

# =============================================================================
# GET FRESH DATA FROM QUERIES MODULE (Snowflake)
# =============================================================================

# 1. Current Cart Rules
df_cart_rules = get_current_cart_rules()

# Merge with main df (by cohort_id + product_id)
if 'cohort_id' in df.columns and len(df_cart_rules) > 0:
    df = df.drop(columns=['current_cart_rule'], errors='ignore')
    df = df.merge(df_cart_rules, on=['cohort_id', 'product_id'], how='left')
    df['current_cart_rule'] = df['current_cart_rule'].fillna(999)
    print(f"✅ Merged cart rules: {len(df)} records")
else:
    df['current_cart_rule'] = df.get('current_cart_rule', 999)

# 2. Current Stocks
df_stocks = get_current_stocks()

# Merge stocks (by warehouse_id + product_id)
if len(df_stocks) > 0:
    df = df.drop(columns=['stocks'], errors='ignore')
    df = df.merge(df_stocks, on=['warehouse_id', 'product_id'], how='left')
    df['stocks'] = df['stocks'].fillna(0)
    print(f"✅ Merged stocks: {len(df)} records")
else:
    df['stocks'] = df.get('stocks', 0)

# 3. Current WAC (Weighted Average Cost)
df_wac = get_current_wac()

# Merge WAC (by warehouse_id + product_id)
if len(df_wac) > 0:
    df = df.drop(columns=['wac_p'], errors='ignore')
    df = df.merge(df_wac, on=['product_id'], how='left')
    df['wac_p'] = df['wac_p'].fillna(0)
    print(f"✅ Merged WAC: {len(df)} records")
else:
    df['wac_p'] = df.get('wac_p', 0)

# 4. Current Prices
df_prices = get_current_prices()

# Merge prices (by cohort_id + product_id)
if len(df_prices) > 0:
    df = df.drop(columns=['current_price'], errors='ignore')
    df = df.merge(df_prices[['cohort_id', 'product_id', 'current_price']], on=['cohort_id', 'product_id'], how='left')
    df['current_price'] = df['current_price'].fillna(0)
    print(f"✅ Merged current prices: {len(df)} records")
else:
    df['current_price'] = df.get('current_price', 0)

print(f"\nCurrent Stock Stats: min={df['stocks'].min():.0f}, max={df['stocks'].max():.0f}, mean={df['stocks'].mean():.1f}")
print(f"Current WAC Stats: min={df['wac_p'].min():.2f}, max={df['wac_p'].max():.2f}, mean={df['wac_p'].mean():.2f}")
print(f"Current Price Stats: min={df['current_price'].min():.2f}, max={df['current_price'].max():.2f}, mean={df['current_price'].mean():.2f}")


Loading data from Snowflake...
Loaded 56870 records from Snowflake

P80 Qty Stats: min=5.0, max=1652.8, mean=12.6
P70 Retailers Stats: min=2.0, max=146.6, mean=3.6
Std Qty Stats: min=0.0, max=2574.8, mean=6.2
Std Retailers Stats: min=0.0, max=76.2, mean=1.4
Fetching current cart rules...
  Loaded 72968 records
✅ Merged cart rules: 56870 records
Fetching current stocks...
  Loaded 1849603 records
✅ Merged stocks: 56870 records
Fetching current WAC...
  Loaded 8147 records
✅ Merged WAC: 56870 records
Fetching current prices...
  Loaded 116393 records
✅ Merged current prices: 56874 records

Current Stock Stats: min=0, max=28023, mean=74.5
Current WAC Stats: min=0.01, max=2082.02, mean=227.57
Current Price Stats: min=0.01, max=2203.25, mean=237.90


In [4]:
# =============================================================================
# CALCULATE NEW WAC (Accounting for Today's Unreflected Purchases)
# =============================================================================
# This calculates a fresh WAC that accounts for today's purchase receipts
# that haven't been reflected in the database yet (to avoid lag)

print("Calculating new WAC from today's purchase receipts...")

# 1. Query today's PRS data (purchases by product_id)
prs_query = '''
WITH prs AS (
    SELECT DISTINCT 
        product_purchased_receipts.purchased_receipt_id,
        products.id AS product_id,
        product_purchased_receipts.basic_unit_count,
        product_purchased_receipts.purchased_item_count * product_purchased_receipts.basic_unit_count AS purchase_min_count,
        product_purchased_receipts.final_price / product_purchased_receipts.purchased_item_count AS final_item_price
    FROM product_purchased_receipts
    LEFT JOIN products ON products.id = product_purchased_receipts.product_id
    LEFT JOIN purchased_receipts ON purchased_receipts.id = product_purchased_receipts.purchased_receipt_id
    WHERE product_purchased_receipts.purchased_item_count <> 0
        AND purchased_receipts.purchased_receipt_status_id IN (4,5,7)
        AND purchased_receipts.date::date >= current_date
        AND product_purchased_receipts.final_price > 0 
)
SELECT 
    product_id,
    SUM(purchase_min_count) AS all_day_qty,
    AVG(final_item_price / basic_unit_count) AS item_price
FROM prs
GROUP BY 1
'''
df_prs = setup_environment_2.dwh_pg_query(prs_query, columns=['product_id', 'all_day_qty', 'item_price'])
df_prs['product_id'] = pd.to_numeric(df_prs['product_id'])
df_prs['all_day_qty'] = pd.to_numeric(df_prs['all_day_qty'])
df_prs['item_price'] = pd.to_numeric(df_prs['item_price'])
print(f"  Loaded {len(df_prs)} PRS records (today's purchases)")

# 2. Query what's already reflected in WAC tracker
reflected_query = '''
SELECT
    t_product_id AS product_id,
    s_beg_stock AS av_stocks,
    p_purchased_item_count AS pr_qty
FROM finance.wac_tracker wt
WHERE wt.t_date::date = CURRENT_DATE
    AND p_purchased_item_count > 0 
'''
try:
    df_reflected = setup_environment_2.dwh_pg_query(reflected_query, columns=['product_id', 'av_stocks', 'pr_qty'])
    df_reflected['product_id'] = pd.to_numeric(df_reflected['product_id'])
    df_reflected['av_stocks'] = pd.to_numeric(df_reflected['av_stocks'])
    df_reflected['pr_qty'] = pd.to_numeric(df_reflected['pr_qty'])
    print(f"  Loaded {len(df_reflected)} WAC tracker records (already reflected)")
except:
    df_reflected = pd.DataFrame(columns=['product_id', 'av_stocks', 'pr_qty'])
    print("  No WAC tracker records found for today")

# 3. Query base WAC (wac1, wac_p) from all_cogs
wac_base_query = '''
SELECT 
    f.product_id,
    f.wac1,
    f.wac_p
FROM finance.all_cogs f
JOIN products ON products.id = f.product_id
JOIN categories ON products.category_id = categories.id
WHERE current_timestamp BETWEEN f.from_date AND f.to_date 
    AND NOT categories.name_ar IN (
        SELECT categories.name_ar AS cat
        FROM categories
        JOIN sections s ON s.id = categories.section_id
        WHERE categories.name_ar LIKE '%سايب%'
            OR categories.name_ar LIKE '%بالتة%'
            OR categories.section_id IN (225, 318, 285, 121, 87, 351, 417)
    )
'''
df_wac_base = query_snowflake(wac_base_query)
df_wac_base['product_id'] = pd.to_numeric(df_wac_base['product_id'])
df_wac_base['wac1'] = pd.to_numeric(df_wac_base['wac1'])
df_wac_base['wac_p'] = pd.to_numeric(df_wac_base['wac_p'])
print(f"  Loaded {len(df_wac_base)} base WAC records from all_cogs")

# 4. Merge and calculate new WAC
df_wac_calc = df_wac_base.merge(df_prs, on='product_id', how='left')
df_wac_calc = df_wac_calc.merge(df_reflected, on='product_id', how='left')
df_wac_calc = df_wac_calc.fillna(0)

# Use current_stock from main df if av_stocks is 0
df_stock_lookup = df[['product_id', 'stocks']].drop_duplicates()
df_stock_lookup = df_stock_lookup.groupby(['product_id'])['stocks'].sum().reset_index()
df_wac_calc = df_wac_calc.merge(df_stock_lookup, on='product_id', how='left')
df_wac_calc['av_stocks'] = df_wac_calc.apply(
    lambda row: row['stocks'] if row['av_stocks'] == 0 else row['av_stocks'], axis=1)

# Calculate not reflected qty (purchases not yet in WAC tracker)
df_wac_calc['not_reflected_qty'] = df_wac_calc['all_day_qty'] - df_wac_calc['pr_qty']
df_wac_calc['not_reflected_qty'] = df_wac_calc['not_reflected_qty'].clip(lower=0)  # Can't be negative

# Calculate new WAC
# new_wac1 = ((stocks + pr_qty) * wac1 + not_reflected_qty * item_price) / (stocks + pr_qty + not_reflected_qty)
denominator = df_wac_calc['av_stocks'] + df_wac_calc['pr_qty'] + df_wac_calc['not_reflected_qty']
numerator = ((df_wac_calc['av_stocks'] + df_wac_calc['pr_qty']) * df_wac_calc['wac1'] + 
             df_wac_calc['not_reflected_qty'] * df_wac_calc['item_price'])

df_wac_calc['new_wac1'] = np.where(denominator > 0, numerator / denominator, df_wac_calc['wac1'])

# Calculate wac1 change and new_wac_p
df_wac_calc['wac1_change'] = np.where(
    df_wac_calc['wac1'] > 0,
    (df_wac_calc['new_wac1'] - df_wac_calc['wac1']) / df_wac_calc['wac1'],
    0
)
df_wac_calc['new_wac_p'] = df_wac_calc['wac_p'] * (1 + df_wac_calc['wac1_change'])
df_wac_calc['new_wac_p'] = df_wac_calc['new_wac_p'].fillna(df_wac_calc['wac_p'])

# Prepare final new_wac dataframe
df_new_wac = df_wac_calc[['product_id','new_wac_p', 'not_reflected_qty']].copy()
df_new_wac=df_new_wac.drop_duplicates()

# 5. Merge new_wac into main dataframe
df = df.drop(columns=['new_wac_p','not_reflected_qty'], errors='ignore')
df = df.merge(df_new_wac, on='product_id', how='left')
df['new_wac'] = df['new_wac_p'].fillna(df['wac_p'])  # Fallback to current_wac if no new_wac

print(f"\n✅ New WAC calculated: {len(df)} records")
print(f"   Products with unreflected purchases: {(df['not_reflected_qty'] > 0).sum()}")
print(f"   New WAC Stats: min={df['new_wac'].min():.2f}, max={df['new_wac'].max():.2f}, mean={df['new_wac'].mean():.2f}")


Calculating new WAC from today's purchase receipts...
  Loaded 338 PRS records (today's purchases)
  Loaded 17 WAC tracker records (already reflected)
  Loaded 7688 base WAC records from all_cogs

✅ New WAC calculated: 56874 records
   Products with unreflected purchases: 5842
   New WAC Stats: min=0.01, max=2082.02, mean=227.57


In [5]:
# =============================================================================
# QUERY 1: TODAY'S UTH (Up-Till-Hour) PERFORMANCE
# =============================================================================
# Gets cumulative qty and retailers from start of day until current hour
# Uses get_uth_performance() from queries_module

df_uth_today = get_uth_performance()


Fetching UTH performance from Snowflake...
  Loaded 7351 UTH records


In [6]:
# =============================================================================
# QUERY 2: TODAY'S LAST HOUR PERFORMANCE (from DWH)
# =============================================================================
# Gets qty and retailers for the PREVIOUS hour only (not cumulative)
# Uses get_last_hour_performance() from queries_module (DWH/PostgreSQL)

df_last_hour = get_last_hour_performance()


Fetching last hour performance from DWH...
  Loaded 1330 last hour records from DWH


In [7]:
# =============================================================================
# QUERY 3: HISTORICAL HOURLY DISTRIBUTION (Last 4 Months) - By Category & Warehouse
# =============================================================================
# Gets:
# - avg_uth_pct_qty/retailers: Average contribution of hours 0 to (current_hour-1) to daily total
# - avg_last_hour_pct_qty/retailers: Average contribution of (current_hour-1) alone to daily total
# Uses get_hourly_distribution() from queries_module

df_hourly_dist = get_hourly_distribution()
print(f"\nAvg UTH % (qty): {df_hourly_dist['avg_uth_pct_qty'].mean()*100:.1f}%")
print(f"Avg Last Hour % (qty): {df_hourly_dist['avg_last_hour_pct_qty'].mean()*100:.1f}%")


Fetching hourly distribution from Snowflake...
  Loaded 771 hourly distribution records

Avg UTH % (qty): 31.8%
Avg Last Hour % (qty): 3.9%


In [8]:
# =============================================================================
# MERGE DATA
# =============================================================================
print("Merging performance data with base data...")

# Merge UTH today data
if len(df_uth_today) > 0:
    df = df.merge(df_uth_today, on=['warehouse_id', 'product_id'], how='left')
else:
    df['uth_qty'] = 0
    df['uth_nmv'] = 0
    df['uth_retailers'] = 0

# Merge last hour data
if len(df_last_hour) > 0:
    df = df.merge(df_last_hour, on=['warehouse_id', 'product_id'], how='left')
else:
    df['last_hour_qty'] = 0
    df['last_hour_nmv'] = 0
    df['last_hour_retailers'] = 0

# Merge hourly distribution (by warehouse_id + cat)
if len(df_hourly_dist) > 0:
    df = df.merge(df_hourly_dist, on=['warehouse_id', 'cat'], how='left')
else:
    df['avg_uth_pct_qty'] = 0.5
    df['avg_uth_pct_retailers'] = 0.5
    df['avg_last_hour_pct_qty'] = 0.05
    df['avg_last_hour_pct_retailers'] = 0.05

# Fill NaN values
df['uth_qty'] = df['uth_qty'].fillna(0)
df['uth_nmv'] = df['uth_nmv'].fillna(0)
df['uth_retailers'] = df['uth_retailers'].fillna(0)
df['last_hour_qty'] = df['last_hour_qty'].fillna(0)
df['last_hour_nmv'] = df['last_hour_nmv'].fillna(0)
df['last_hour_retailers'] = df['last_hour_retailers'].fillna(0)
df['avg_uth_pct_qty'] = df['avg_uth_pct_qty'].fillna(0.5)
df['avg_uth_pct_retailers'] = df['avg_uth_pct_retailers'].fillna(0.5)
df['avg_last_hour_pct_qty'] = df['avg_last_hour_pct_qty'].fillna(0.05)
df['avg_last_hour_pct_retailers'] = df['avg_last_hour_pct_retailers'].fillna(0.05)

print(f"✅ Merged data: {len(df)} records")
print(f"\nUTH Qty Stats: min={df['uth_qty'].min():.0f}, max={df['uth_qty'].max():.0f}, mean={df['uth_qty'].mean():.1f}")
print(f"Last Hour Qty Stats: min={df['last_hour_qty'].min():.0f}, max={df['last_hour_qty'].max():.0f}, mean={df['last_hour_qty'].mean():.1f}")
print(f"Current Cart Rule Stats: min={df['current_cart_rule'].min():.0f}, max={df['current_cart_rule'].max():.0f}, mean={df['current_cart_rule'].mean():.1f}")


Merging performance data with base data...
✅ Merged data: 56874 records

UTH Qty Stats: min=0, max=772, mean=2.3
Last Hour Qty Stats: min=0, max=125, mean=0.2
Current Cart Rule Stats: min=2, max=626, mean=20.9


In [9]:
# =============================================================================
# CALCULATE TARGETS AND STATUSES
# =============================================================================
print("Calculating UTH and Last Hour targets and statuses...")

def get_status_std(actual, target, std, is_qty=True):
    """
    Determine status based on ±3 std from target.
    - Growing: actual > target + 3*std
    - On Track: target - 3*std <= actual <= target + 3*std
    - Dropping: actual < target - 3*std (minimum threshold = 1)
    
    Parameters:
    - actual: actual performance value
    - target: target value (p80 for qty, p70 for retailers)
    - std: standard deviation (std_daily_240d or std_daily_retailers_240d)
    - is_qty: True for qty metrics, False for retailer metrics
    """
    upper_bound = target + STD_THRESHOLD * std
    lower_bound = max(target - STD_THRESHOLD * std, MIN_DROPPING_THRESHOLD)
    
    if actual > upper_bound:
        return 'growing'
    elif actual < lower_bound:
        return 'dropping'
    else:
        return 'on_track'

# Calculate UTH targets
# UTH target = p80_qty * avg_uth_pct (historical % of day that should be done by now)
df['uth_qty_target'] = df['p80_daily_240d'] * df['avg_uth_pct_qty']
df['uth_rets_target'] = df['p70_daily_retailers_240d'] * df['avg_uth_pct_retailers']

# Calculate UTH std thresholds (scaled by UTH percentage)
df['uth_qty_std'] = df['std_daily_240d'] * df['avg_uth_pct_qty']
df['uth_rets_std'] = df['std_daily_retailers_240d'] * df['avg_uth_pct_retailers']

# Calculate Last Hour targets
# Last hour target = p80_qty * avg_last_hour_pct (historical % of day for this hour)
df['last_hour_qty_target'] = df['p80_daily_240d'] * df['avg_last_hour_pct_qty']
df['last_hour_rets_target'] = df['p70_daily_retailers_240d'] * df['avg_last_hour_pct_retailers']

# Calculate Last Hour std thresholds (scaled by last hour percentage)
df['last_hour_qty_std'] = df['std_daily_240d'] * df['avg_last_hour_pct_qty']
df['last_hour_rets_std'] = df['std_daily_retailers_240d'] * df['avg_last_hour_pct_retailers']

# Calculate statuses using ±3 std thresholds
df['uth_qty_status'] = df.apply(
    lambda row: get_status_std(row['uth_qty'], row['uth_qty_target'], row['uth_qty_std'], is_qty=True), axis=1)
df['uth_rets_status'] = df.apply(
    lambda row: get_status_std(row['uth_retailers'], row['uth_rets_target'], row['uth_rets_std'], is_qty=False), axis=1)
df['last_hour_qty_status'] = df.apply(
    lambda row: get_status_std(row['last_hour_qty'], row['last_hour_qty_target'], row['last_hour_qty_std'], is_qty=True), axis=1)
df['last_hour_rets_status'] = df.apply(
    lambda row: get_status_std(row['last_hour_retailers'], row['last_hour_rets_target'], row['last_hour_rets_std'], is_qty=False), axis=1)

print(f"✅ Targets and statuses calculated using ±{STD_THRESHOLD} std thresholds")

# Summary
print(f"\n{'='*60}")
print("UTH STATUS DISTRIBUTION")
print(f"{'='*60}")
print(f"\nUTH Qty Status:")
print(df['uth_qty_status'].value_counts().to_string())
print(f"\nUTH Retailers Status:")
print(df['uth_rets_status'].value_counts().to_string())

print(f"\n{'='*60}")
print("LAST HOUR STATUS DISTRIBUTION")
print(f"{'='*60}")
print(f"\nLast Hour Qty Status:")
print(df['last_hour_qty_status'].value_counts().to_string())
print(f"\nLast Hour Retailers Status:")
print(df['last_hour_rets_status'].value_counts().to_string())


Calculating UTH and Last Hour targets and statuses...
✅ Targets and statuses calculated using ±3 std thresholds

UTH STATUS DISTRIBUTION

UTH Qty Status:
uth_qty_status
dropping    43354
on_track    10982
growing      2538

UTH Retailers Status:
uth_rets_status
dropping    42892
on_track    10194
growing      3788

LAST HOUR STATUS DISTRIBUTION

Last Hour Qty Status:
last_hour_qty_status
dropping    54386
growing      1368
on_track     1120

Last Hour Retailers Status:
last_hour_rets_status
dropping    54386
growing      1814
on_track      674


In [10]:
# =============================================================================
# FETCH MARKET DATA AND MARGIN TIERS
# =============================================================================
print("Fetching market data and margin tiers...")

# Import market_data_module for get_market_data()
%run market_data_module.ipynb

# 1. Get margin tiers (already available from queries_module)
df_margin_tiers = get_margin_tiers()
print(f"✅ Loaded {len(df_margin_tiers)} margin tier records")

# 2. Get market data
df_market = get_market_data()
print(f"✅ Loaded {len(df_market)} market data records")

# Verify cohort_id is available in all dataframes
print(f"\nChecking cohort_id availability:")
print(f"  - df has cohort_id: {'cohort_id' in df.columns}")
print(f"  - df_margin_tiers has cohort_id: {'cohort_id' in df_margin_tiers.columns}")
print(f"  - df_market has cohort_id: {'cohort_id' in df_market.columns}")

# 3. Merge margin tiers with df (by cohort_id + product_id) - ALL COLUMNS
merge_keys = ['cohort_id', 'product_id']
df = df.drop(columns=[c for c in df_margin_tiers.columns if c in df.columns and c not in merge_keys], errors='ignore')
df = df.merge(df_margin_tiers, on=merge_keys, how='left')
print(f"\n✅ Merged margin tiers: {len(df)} records")
print(f"   Margin tier columns added: {[c for c in df_margin_tiers.columns if c not in merge_keys]}")

# 4. Merge market data with df (by cohort_id + product_id) - ALL COLUMNS
df = df.drop(columns=[c for c in df_market.columns if c in df.columns and c not in merge_keys], errors='ignore')
df = df.merge(df_market, on=merge_keys, how='left')
print(f"✅ Merged market data: {len(df)} records")
print(f"   Market columns added: {[c for c in df_market.columns if c not in merge_keys]}")

# 5. Calculate current margin based on current_price and new_wac
df['current_margin'] = (df['current_price'] - df['new_wac']) / df['current_price']
df['current_margin'] = df['current_margin'].fillna(0)

print(f"\nMargin Stats: min={df['current_margin'].min():.2%}, max={df['current_margin'].max():.2%}, mean={df['current_margin'].mean():.2%}")

# =============================================================================
# FILTER SKUS FOR ACTION
# =============================================================================
# Filter SKUs that need action based on:
# Pre-condition: below_min_stock_flag != 1 (must be sellable)
# Condition A: new_wac > wac_p * 1.005 (WAC increased by more than 0.5%)
# Condition B: Last hour growing (qty OR rets) AND UTH (one growing + other on_track/growing) AND last_hour_qty > 5

print("Filtering SKUs for action...")

# Skip SKUs with below_min_stock_flag (stock < min selling unit qty)
can_sell = df['below_min_stock_flag'].fillna(0) != 1

# Condition A: WAC increased significantly
condition_a = (df['new_wac'] > df['wac_p'] * 1.005)

# Condition B: Last hour growing AND UTH performing well AND last_hour_qty > 5
# UTH good = one must be 'growing' AND the other must be 'on_track' or 'growing'
last_hour_growing = (df['last_hour_qty_status'] == 'growing') | (df['last_hour_rets_status'] == 'growing')
uth_good = (
    ((df['uth_qty_status'] == 'growing') & (df['uth_rets_status'].isin(['on_track', 'growing']))) |
    ((df['uth_rets_status'] == 'growing') & (df['uth_qty_status'].isin(['on_track', 'growing'])))
)
last_hour_qty_threshold = df['last_hour_qty'] > 5
condition_b = last_hour_growing & uth_good & last_hour_qty_threshold & can_sell

# Combine conditions (A OR B)
df['needs_action'] = condition_a | condition_b
df['action_reason'] = 'none'
df.loc[condition_a & ~condition_b, 'action_reason'] = 'wac_increase'
df.loc[~condition_a & condition_b, 'action_reason'] = 'growing_performance'
df.loc[condition_a & condition_b, 'action_reason'] = 'both'

# Define important columns for df_action
action_columns = [
    # Identification
    'warehouse_id', 'product_id', 'cohort_id', 'sku', 'region', 'brand', 'cat',
    # Price & Cost
    'current_price', 'wac_p', 'new_wac', 'current_margin',
    # Inventory & Rules
    'stocks', 'current_cart_rule',
    # Performance Targets
    'p80_daily_240d', 'p70_daily_retailers_240d', 'std_daily_240d', 'std_daily_retailers_240d',
    # Refill columns for cart rule calculation
    'normal_refill', 'refill_stddev', 'target_margin',
    # UTH Status
    'uth_qty', 'uth_qty_status', 'uth_retailers', 'uth_rets_status',
    # Last Hour Status
    'last_hour_qty', 'last_hour_qty_status', 'last_hour_retailers', 'last_hour_rets_status',
    # Market Margins
    'below_market', 'market_min', 'market_25', 'market_50', 'market_75', 'market_max', 'above_market',
    # Margin Tiers (excluding: effective_min_margin, max_boundary, margin_step, margin_tier_below)
    'margin_tier_1', 'margin_tier_2', 'margin_tier_3', 'margin_tier_4', 'margin_tier_5',
    'margin_tier_above_1', 'margin_tier_above_2',
    # Action Flags
    'needs_action', 'action_reason'
]

# Filter to only columns that exist in df
action_columns = [c for c in action_columns if c in df.columns]

# Filter to action SKUs with selected columns only
df_action = df[df['needs_action']][action_columns].copy()

print(f"\n{'='*60}")
print("SKUs FILTERED FOR ACTION")
print(f"{'='*60}")
print(f"\nTotal SKUs needing action: {len(df_action)} out of {len(df)} ({len(df_action)/len(df)*100:.1f}%)")
print(f"\nBreakdown by reason:")
print(df_action['action_reason'].value_counts().to_string())

print(f"\n--- Condition A (WAC increase > 0.5%): {condition_a.sum()} SKUs")
print(f"--- Condition B (Last hour growing + UTH good + qty > 5): {condition_b.sum()} SKUs")

# Show sample
print(f"\n{'='*60}")
print("SAMPLE ACTION SKUs")
print(f"{'='*60}")
print(f"\nTotal columns in df_action: {len(df_action.columns)}")
print(f"Columns: {df_action.columns.tolist()}")
display(df_action.head(10))


Fetching market data and margin tiers...
/home/ec2-user/.Renviron
/home/ec2-user/service_account_key.json
Market Data Module loaded at 2026-01-27 13:21:02 Cairo time
Snowflake timezone: America/Los_Angeles
All queries defined ✓
Helper functions defined ✓
get_market_data() function defined ✓
get_margin_tiers() function defined ✓

MARKET DATA MODULE READY

Available functions (NO INPUT REQUIRED):
  - get_market_data()   : Fetch and process all market prices
  - get_margin_tiers()  : Fetch and calculate margin tiers

Usage:
  %run market_data_module.ipynb
  df_market = get_market_data()
  df_tiers = get_margin_tiers()

FETCHING MARGIN TIERS
Timestamp: 2026-01-27 13:21:03 Cairo time

Step 1: Fetching margin boundaries from PRODUCT_STATISTICS...
    Loaded 18328 records

Step 2: Adding cohort IDs...
    Records with cohorts: 25258

Step 3: Calculating margin tiers...

MARGIN TIERS COMPLETE
Total records: 25258

Margin Tier Structure:
  margin_tier_below:   effective_min - step (1 below)
  m

/tmp/ipykernel_23502/1916930114.py:138: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


    Records after group processing: 25629

Step 5: Adding WAC and margin data...
    Records with WAC: 25413

Step 6: Filtering by price coverage...
    Records after price coverage filter: 13496

Step 7: Calculating price percentiles...
    Records after price analysis: 12777

Step 8: Converting prices to margins...

MARKET DATA COMPLETE
Total records: 12777
  - With marketplace prices: 12204
  - With scrapped prices: 6203
  - With Ben Soliman prices: 8663
✅ Loaded 12777 market data records

Checking cohort_id availability:
  - df has cohort_id: True
  - df_margin_tiers has cohort_id: True
  - df_market has cohort_id: True

✅ Merged margin tiers: 56874 records
   Margin tier columns added: ['region', 'optimal_bm', 'min_boundary', 'max_boundary', 'median_bm', 'effective_min_margin', 'margin_step', 'margin_tier_below', 'margin_tier_1', 'margin_tier_2', 'margin_tier_3', 'margin_tier_4', 'margin_tier_5', 'margin_tier_above_1', 'margin_tier_above_2']
✅ Merged market data: 56874 records
   

,warehouse_id,product_id,cohort_id,sku,region,brand,cat,current_price,wac_p,new_wac,...,above_market,margin_tier_1,margin_tier_2,margin_tier_3,margin_tier_4,margin_tier_5,margin_tier_above_1,margin_tier_above_2,needs_action,action_reason
366,962,1776,701,عصير بست كوكتيل كيس - 190 مل,Giza,بست,عصاير,265.00,250.917804,250.917804,...,0.128253,0.077240,0.090502,0.103765,0.117028,0.130290,0.143553,0.156815,True,growing_performance
1177,703,11901,1123,أد- مى مربى تين - 360 جم,NaN,اد-مي,مربي,313.25,302.749225,305.986717,...,NaN,0.033447,0.043003,0.052560,0.062116,0.071672,0.081228,0.090785,True,wac_increase
2290,339,2054,704,كابتشينو كوفى بريك موكا - 18.5 جم,Delta East,كوفي بريك,قهوة,61.25,58.714552,58.714552,...,0.145970,0.034597,0.042815,0.051033,0.059251,0.067469,0.075688,0.083906,True,growing_performance
2326,1,21946,700,بقسماط ناعم ريتش بيك - 400 جرام,Cairo,ريتش بيك,مرقة وخلطات,395.00,339.127156,345.258102,...,0.236200,0.055007,0.067104,0.079202,0.091299,0.103396,0.115493,0.127591,True,wac_increase
2353,236,1622,701,شامبو سباركل مشكل 30جنية كميه أكبر - 40 شريط,Giza,سباركل,شامبو و شاور جيل,278.75,262.808050,262.808050,...,0.141618,0.026600,0.033916,0.041232,0.048548,0.055864,0.063180,0.070496,True,growing_performance
2779,962,6097,701,حفاضات جود كير مقاس 5 - 100 حفاضة,Giza,جود كير,حفاضات أطفال,423.50,390.303988,390.287749,...,0.126055,0.050195,0.056396,0.062597,0.068799,0.075000,0.081201,0.087403,True,growing_performance
3137,962,8649,701,شويبس يوسفى - 950 مل,Giza,شويبس,حاجه ساقعه,135.00,127.964287,127.964287,...,0.181277,0.077398,0.085463,0.093529,0.101594,0.109659,0.117725,0.125790,True,growing_performance
4966,401,2805,1126,مسحوق غسيل فل ازرق يدوى - 9 كجم,Upper Egypt,فل,منظفات,155.25,140.810502,140.821424,...,0.251642,0.064850,0.079261,0.093672,0.108083,0.122494,0.136904,0.151315,True,growing_performance
5018,797,20927,702,اندومي سوبر مى خضار حارجامبو عرض 44 كيس - 100 جم,Alexandria,اندومي,شعرية سريعة التحضير,361.25,354.380000,354.380000,...,0.084132,0.025316,0.031487,0.037658,0.043829,0.050000,0.056171,0.062342,True,growing_performance
5755,703,10588,1123,بسكاتو دايجستا ميني لايت - 8 جنية,NaN,بسكاتو,بسكويت و معمول,70.00,67.004311,67.739238,...,NaN,0.049000,0.059539,0.070078,0.080617,0.091156,0.101694,0.112233,True,wac_increase


In [11]:
# =============================================================================
# ACTION LOGIC
# =============================================================================
print("Calculating actions for filtered SKUs...")

# Step 0: Preparation - Calculate normal_refill_3std
df_action['normal_refill_3std'] = df_action['normal_refill'] + 3 * df_action['refill_stddev']

# Initialize output columns
df_action['new_price'] = np.nan
df_action['new_cart_rule'] = np.nan
df_action['price_action'] = 'none'
df_action['cart_rule_action'] = 'none'

# =============================================================================
# HELPER: Calculate all price tiers from margins using wac_p
# =============================================================================
def calculate_price_from_margin(wac, margin):
    """Calculate price from WAC and margin: price = wac / (1 - margin)"""
    if pd.isna(margin) or margin >= 1:
        return np.nan
    return wac / (1 - margin)

# Calculate market prices (using wac_p)
market_margin_cols = ['market_min', 'market_25', 'market_50', 'market_75', 'market_max']
for col in market_margin_cols:
    price_col = f'{col}_price'
    df_action[price_col] = df_action.apply(
        lambda row: calculate_price_from_margin(row['wac_p'], row[col]), axis=1)

# Calculate margin tier prices (using wac_p)
tier_margin_cols = ['margin_tier_1', 'margin_tier_2', 'margin_tier_3', 'margin_tier_4', 'margin_tier_5',
                    'margin_tier_above_1', 'margin_tier_above_2']
for col in tier_margin_cols:
    price_col = f'{col}_price'
    df_action[price_col] = df_action.apply(
        lambda row: calculate_price_from_margin(row['wac_p'], row[col]), axis=1)

# =============================================================================
# HELPER: Find first price above threshold
# =============================================================================
def find_first_price_above(row, threshold, price_cols):
    """Find the first price in price_cols that is above threshold."""
    for col in price_cols:
        price = row.get(col, np.nan)
        if pd.notna(price) and price > threshold:
            return price
    return np.nan

# Define price column order (market first, then tiers)
market_price_cols = ['market_min_price', 'market_25_price', 'market_50_price', 'market_75_price', 'market_max_price']
tier_price_cols = ['margin_tier_1_price', 'margin_tier_2_price', 'margin_tier_3_price', 
                   'margin_tier_4_price', 'margin_tier_5_price', 
                   'margin_tier_above_1_price', 'margin_tier_above_2_price']
all_price_cols = market_price_cols + tier_price_cols

# =============================================================================
# STEP 1: WAC Increase Actions (action_reason = 'both' or 'wac_increase')
# =============================================================================
# Only apply if: current_price < new_wac AND current_price > wac_p
# (price is squeezed between old and new WAC)
wac_increase_mask = (
    df_action['action_reason'].isin(['both', 'wac_increase']) &
    (df_action['current_price'] < df_action['new_wac']) &
    (df_action['current_price'] > df_action['wac_p'])
)

def get_wac_increase_price(row):
    """Find first market/tier price above current_price."""
    # Find first price above current_price (not new_wac)
    new_price = find_first_price_above(row, row['current_price'], all_price_cols)
    return new_price

df_action.loc[wac_increase_mask, 'new_price'] = df_action[wac_increase_mask].apply(get_wac_increase_price, axis=1)
df_action.loc[wac_increase_mask, 'price_action'] = 'wac_increase'

print(f"  WAC increase actions: {wac_increase_mask.sum()} SKUs")
print(f"    (SKUs where wac_p < current_price < new_wac)")

# =============================================================================
# STEP 2: Growing Performance Actions (action_reason = 'growing_performance')
# =============================================================================
growing_mask = df_action['action_reason'] == 'growing_performance'

# Case A: Retailers growing (with or without qty)
rets_growing_mask = growing_mask & (df_action['last_hour_rets_status'] == 'growing')

def get_rets_growing_price(row):
    """Find price for rets growing: market/tier >= current_price * 1.005, or fallback to target margin."""
    min_price = row['current_price'] * 1.005
    
    # Try market prices first
    new_price = find_first_price_above(row, min_price, market_price_cols)
    if pd.notna(new_price):
        return new_price
    
    # Try tier prices
    new_price = find_first_price_above(row, min_price, tier_price_cols)
    if pd.notna(new_price):
        return new_price
    
    # Fallback: wac_p / (1 - (target_margin * 1.15))
    target_margin = row.get('target_margin', 0)
    if pd.notna(target_margin) and target_margin * 1.15 < 1:
        fallback_price = row['wac_p'] / (1 - (target_margin * 1.15))
        # If fallback price is lower than current price, use current_price * 1.01
        if fallback_price < row['current_price']:
            return row['current_price'] * 1.01
        return fallback_price
    
    return np.nan

df_action.loc[rets_growing_mask, 'new_price'] = df_action[rets_growing_mask].apply(get_rets_growing_price, axis=1)
df_action.loc[rets_growing_mask, 'price_action'] = 'rets_growing'

print(f"  Rets growing actions (price increase): {rets_growing_mask.sum()} SKUs")

# Case B: Qty growing only (rets NOT growing)
qty_only_growing_mask = growing_mask & (df_action['last_hour_qty_status'] == 'growing') & (df_action['last_hour_rets_status'] != 'growing')

def get_qty_growing_cart_rule(row):
    """Calculate new cart rule for qty growing only."""
    # Calculate qty per retailer
    uth_retailers = row.get('uth_retailers', 0)
    if uth_retailers == 0:
        qty_per_retailer = np.inf
    else:
        qty_per_retailer = row['uth_qty'] / uth_retailers
    
    # Get the three options
    option1 = row.get('normal_refill_3std', np.inf)
    option2 = qty_per_retailer
    option3 = row.get('current_cart_rule', np.inf) * 0.85
    
    # Return minimum of the three (excluding inf/nan)
    options = [opt for opt in [option1, option2, option3] if pd.notna(opt) and opt != np.inf]
    if options:
        return min(options)
    return np.nan

df_action.loc[qty_only_growing_mask, 'new_cart_rule'] = df_action[qty_only_growing_mask].apply(get_qty_growing_cart_rule, axis=1)

# Round and apply min/max constraints to new_cart_rule
df_action['new_cart_rule'] = df_action['new_cart_rule'].round()
df_action['new_cart_rule'] = df_action['new_cart_rule'].clip(lower=2, upper=150)

df_action.loc[qty_only_growing_mask, 'cart_rule_action'] = 'qty_growing'

print(f"  Qty growing actions (cart rule change): {qty_only_growing_mask.sum()} SKUs")
print(f"    (Cart rule: rounded, min=2, max=150)")

# =============================================================================
# SUMMARY
# =============================================================================
print(f"\n{'='*60}")
print("ACTION SUMMARY")
print(f"{'='*60}")
print(f"\nPrice Actions:")
print(df_action['price_action'].value_counts().to_string())
print(f"\nCart Rule Actions:")
print(df_action['cart_rule_action'].value_counts().to_string())

# Show SKUs with new prices
price_changes = df_action[df_action['new_price'].notna()]
print(f"\nSKUs with new price: {len(price_changes)}")
if len(price_changes) > 0:
    print(f"  Avg price change: {((price_changes['new_price'] - price_changes['current_price']) / price_changes['current_price']).mean()*100:.1f}%")

# Show SKUs with new cart rules
cart_changes = df_action[df_action['new_cart_rule'].notna()]
print(f"\nSKUs with new cart rule: {len(cart_changes)}")
if len(cart_changes) > 0:
    print(f"  Avg cart rule change: {((cart_changes['new_cart_rule'] - cart_changes['current_cart_rule']) / cart_changes['current_cart_rule']).mean()*100:.1f}%")

# Display sample
print(f"\n{'='*60}")
print("SAMPLE OUTPUT")
print(f"{'='*60}")
output_cols = ['sku', 'current_price', 'new_price', 'current_cart_rule', 'new_cart_rule', 
               'price_action', 'cart_rule_action', 'action_reason']
output_cols = [c for c in output_cols if c in df_action.columns]
display(df_action[df_action['new_price'].notna() | df_action['new_cart_rule'].notna()][output_cols].head(15))


Calculating actions for filtered SKUs...
  WAC increase actions: 4 SKUs
    (SKUs where wac_p < current_price < new_wac)
  Rets growing actions (price increase): 92 SKUs
  Qty growing actions (cart rule change): 10 SKUs
    (Cart rule: rounded, min=2, max=150)

ACTION SUMMARY

Price Actions:
price_action
none            128
rets_growing     92
wac_increase      4

Cart Rule Actions:
cart_rule_action
none           214
qty_growing     10

SKUs with new price: 96
  Avg price change: 3.0%

SKUs with new cart rule: 10
  Avg cart rule change: -82.1%

SAMPLE OUTPUT


,sku,current_price,new_price,current_cart_rule,new_cart_rule,price_action,cart_rule_action,action_reason
366,عصير بست كوكتيل كيس - 190 مل,265.00,267.000000,13,NaN,rets_growing,none,growing_performance
2290,كابتشينو كوفى بريك موكا - 18.5 جم,61.25,65.000000,20,NaN,rets_growing,none,growing_performance
2353,شامبو سباركل مشكل 30جنية كميه أكبر - 40 شريط,278.75,285.000000,150,NaN,rets_growing,none,growing_performance
2779,حفاضات جود كير مقاس 5 - 100 حفاضة,423.50,426.700000,80,NaN,rets_growing,none,growing_performance
3137,شويبس يوسفى - 950 مل,135.00,143.257500,18,NaN,rets_growing,none,growing_performance
4966,مسحوق غسيل فل ازرق يدوى - 9 كجم,155.25,158.750000,38,NaN,rets_growing,none,growing_performance
5018,اندومي سوبر مى خضار حارجامبو عرض 44 كيس - 100 جم,361.25,365.600006,14,NaN,rets_growing,none,growing_performance
6233,كوكا كولا تربو زيرو - 300 مل,78.75,NaN,47,8.0,none,qty_growing,growing_performance
6268,بقسماط ناعم ريتش بيك - 400 جرام,343.50,352.000000,25,NaN,wac_increase,none,wac_increase
6327,مسحوق غسيل فل يدوى ابيض - 9 كجم,158.25,160.046096,8,NaN,rets_growing,none,growing_performance


In [12]:
# =============================================================================
# PUSH CONFIGURATION
# =============================================================================
# Mode: 'testing' = prepare files only, 'live' = actually upload to API
PUSH_MODE = 'live'  # Change to 'live' when ready to push

print(f"\n{'='*60}")
print(f"PUSH MODE: {PUSH_MODE.upper()}")
print(f"{'='*60}")
if PUSH_MODE == 'testing':
    print("⚠️ Testing mode - files will be prepared but NOT uploaded")
else:
    print("🚀 Live mode - files will be uploaded to MaxAB API")



PUSH MODE: LIVE
🚀 Live mode - files will be uploaded to MaxAB API


In [13]:
df_action.to_excel("check.xlsx")

In [14]:
# =============================================================================
# IMPORT PUSH HANDLERS
# =============================================================================
%run push_cart_rules_handler.ipynb
%run push_prices_handler.ipynb

print("✅ Push handlers loaded")


Push Cart Rules Handler loaded at 2026-01-27 13:32:09 Cairo time
✓ API credentials loaded successfully
Push Prices Handler loaded at 2026-01-27 13:32:09 Cairo time
✓ API credentials loaded successfully
✓ Google Sheets client initialized
✅ Push handlers loaded


In [15]:
# =============================================================================
# PREPARE DATA FOR PUSH
# =============================================================================
# Get packing units for push handlers
pus = get_packing_units()
print(f"✅ Loaded {len(pus)} packing units")

# Prepare df_action with required columns for push functions
# Prices need: product_id, sku, new_price, warehouse_id, cohort_id, stocks, current_price
# Cart rules need: product_id, sku, new_cart_rule, warehouse_id, cohort_id, stocks, current_cart_rule

# Rename stocks column if needed
if 'current_stocks' in df_action.columns and 'stocks' not in df_action.columns:
    df_action['stocks'] = df_action['current_stocks']

# Summary of what will be pushed
price_changes = df_action[df_action['new_price'].notna() & (df_action['new_price'] != df_action['current_price'])]
cart_changes = df_action[df_action['new_cart_rule'].notna() & (df_action['new_cart_rule'] != df_action['current_cart_rule'])]

print(f"\n📊 Push Summary:")
print(f"  Price changes: {len(price_changes)} SKUs")
print(f"  Cart rule changes: {len(cart_changes)} SKUs")
print(f"  Mode: {PUSH_MODE}")


Fetching packing_units ...
  Loaded 34872 records
✅ Loaded 34872 packing units

📊 Push Summary:
  Price changes: 96 SKUs
  Cart rule changes: 10 SKUs
  Mode: live


In [16]:
# =============================================================================
# STEP 1: PUSH CART RULES
# =============================================================================
# Push cart rules first - if any cohorts fail, we'll skip them for prices too
print(f"\n{'='*60}")
print("STEP 1: PUSH CART RULES")
print(f"{'='*60}")

cart_result = push_cart_rules(df_action, pus, source_module='module_4', mode=PUSH_MODE)

# Track failed cohorts to skip in price push
failed_cohorts = cart_result.get('failed_cohorts', [])

print(f"\n📊 Cart Rules Push Result:")
print(f"  Total received: {cart_result['total_received']}")
print(f"  Cart rule changes: {cart_result['cart_rule_changes']}")
print(f"  Pushed: {cart_result['pushed']}")
print(f"  Skipped: {cart_result['skipped']}")
print(f"  Failed: {cart_result['failed']}")
print(f"  Mode: {cart_result['mode']}")

if failed_cohorts:
    print(f"  ⚠️ Failed cohorts: {failed_cohorts}")



STEP 1: PUSH CART RULES

🚀 MODE: LIVE
   Files will be prepared AND uploaded to API

PUSH CART RULES - Source: module_4
Total received: 224
Cart rule changes to push: 10
Skipped (no change): 214

Cart rule changes summary:
  Increases: 0
  Decreases: 10

📋 Prepared 6 packing unit cart rules

Sample cart rule adjustments (showing products with multiple PUs):
 product_id  basic_unit_count  final_cart_rule  final_pu_cart_rule
       5812                 1                8                   8
       5812                12                8                   2

Processing cohort: 700
  Saved: uploads/module_4_cart_rules_700.xlsx (2 rows)
  Split into 1 chunks (size: 4000)


  Saving chunks: 100%|██████████| 1/1 [00:00<00:00, 233.60it/s]

  Uploading...


    ✓ Chunk 1 uploaded successfully

Processing cohort: 701
  Saved: uploads/module_4_cart_rules_701.xlsx (4 rows)
  Split into 1 chunks (size: 4000)


  Saving chunks: 100%|██████████| 1/1 [00:00<00:00, 253.83it/s]

  Uploading...


    ✓ Chunk 1 uploaded successfully

🚀 UPLOAD COMPLETE
Mode: live
Total prepared: 6
Total failed: 0

📊 Cart Rules Push Result:
  Total received: 224
  Cart rule changes: 10
  Pushed: 6
  Skipped: 214
  Failed: 0
  Mode: live


In [17]:
# =============================================================================
# STEP 2: PUSH PRICES (skip failed cohorts from cart rules)
# =============================================================================
print(f"\n{'='*60}")
print("STEP 2: PUSH PRICES")
print(f"{'='*60}")

# Push prices, skipping any cohorts that failed during cart rules push
push_result = push_prices(df_action, pus, source_module='module_4', mode=PUSH_MODE, skip_cohorts=failed_cohorts)

print(f"\n📊 Prices Push Result:")
print(f"  Total received: {push_result['total_received']}")
print(f"  Price changes: {push_result['price_changes']}")
print(f"  Pushed: {push_result['pushed']}")
print(f"  Skipped: {push_result['skipped']}")
print(f"  Failed: {push_result['failed']}")
print(f"  Mode: {push_result['mode']}")

if push_result.get('skipped_cohorts'):
    print(f"  ⚠️ Skipped cohorts: {push_result['skipped_cohorts']}")



STEP 2: PUSH PRICES

🚀 MODE: LIVE
   Files will be prepared AND uploaded to API
Loading disable_pu_visibility from Google Sheets...
  ✓ Loaded 89 products to disable min PU visibility

PUSH PRICES - Source: module_4
Total received: 224
Price changes to push: 96
Skipped (no change): 128

Price changes summary:
  Increases: 96
  Decreases: 0

📋 Prepared 66 packing unit prices

Processing cohort: 700
  Saved: uploads/module_4_700.xlsx (7 rows)
  Split into 1 chunks (size: 4000)


  Saving chunks: 100%|██████████| 1/1 [00:00<00:00, 208.52it/s]

  Uploading...


    ✓ Chunk 1 uploaded successfully

Processing cohort: 703
  Saved: uploads/module_4_703.xlsx (11 rows)
  Split into 1 chunks (size: 4000)


  Saving chunks: 100%|██████████| 1/1 [00:00<00:00, 184.94it/s]

...

  Uploading...


    ✓ Chunk 1 uploaded successfully

Processing cohort: 1123
  Saved: uploads/module_4_1123.xlsx (2 rows)
  Split into 1 chunks (size: 4000)


  Saving chunks: 100%|██████████| 1/1 [00:00<00:00, 233.11it/s]

  Uploading...


    ✓ Chunk 1 uploaded successfully

🚀 UPLOAD COMPLETE
Mode: live
Total prepared: 66
Total failed: 0

📊 Prices Push Result:
  Total received: 224
  Price changes: 96
  Pushed: 66
  Skipped: 128
  Failed: 0
  Mode: live


In [18]:
# =============================================================================
# FINAL SUMMARY
# =============================================================================
print(f"\n{'='*60}")
print("MODULE 4 - HOURLY UPDATES COMPLETE")
print(f"{'='*60}")
print(f"\n📅 Timestamp: {datetime.now(CAIRO_TZ).strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔧 Mode: {PUSH_MODE.upper()}")

print(f"\n📊 ACTIONS TAKEN:")
print(f"  Total SKUs analyzed: {len(df)}")
print(f"  SKUs requiring action: {len(df_action)}")

# Price actions breakdown
print(f"\n💰 PRICE ACTIONS:")
if 'price_action' in df_action.columns:
    price_actions = df_action[df_action['price_action'] != 'none']
    print(f"  Total price changes: {len(price_actions)}")
    for action_type in df_action['price_action'].unique():
        if action_type != 'none':
            count = len(df_action[df_action['price_action'] == action_type])
            print(f"    - {action_type}: {count}")
else:
    print(f"  Total price changes: 0")

# Cart rule actions breakdown  
print(f"\n🛒 CART RULE ACTIONS:")
if 'cart_rule_action' in df_action.columns:
    cart_actions = df_action[df_action['cart_rule_action'] != 'none']
    print(f"  Total cart rule changes: {len(cart_actions)}")
    for action_type in df_action['cart_rule_action'].unique():
        if action_type != 'none':
            count = len(df_action[df_action['cart_rule_action'] == action_type])
            print(f"    - {action_type}: {count}")
else:
    print(f"  Total cart rule changes: 0")

# Push results
print(f"\n📤 PUSH RESULTS:")
print(f"  Cart Rules - Pushed: {cart_result.get('pushed', 0)}, Failed: {cart_result.get('failed', 0)}")
print(f"  Prices - Pushed: {push_result.get('pushed', 0)}, Failed: {push_result.get('failed', 0)}")

if PUSH_MODE == 'testing':
    print(f"\n⚠️ TESTING MODE - No changes were actually pushed to production!")
    print(f"   Change PUSH_MODE to 'live' to execute actual pushes.")
else:
    print(f"\n✅ LIVE MODE - Changes have been pushed to production!")



MODULE 4 - HOURLY UPDATES COMPLETE

📅 Timestamp: 2026-01-27 13:33:17
🔧 Mode: LIVE

📊 ACTIONS TAKEN:
  Total SKUs analyzed: 56874
  SKUs requiring action: 224

💰 PRICE ACTIONS:
  Total price changes: 96
    - rets_growing: 92
    - wac_increase: 4

🛒 CART RULE ACTIONS:
  Total cart rule changes: 10
    - qty_growing: 10

📤 PUSH RESULTS:
  Cart Rules - Pushed: 6, Failed: 0
  Prices - Pushed: 66, Failed: 0

✅ LIVE MODE - Changes have been pushed to production!


In [19]:
# =============================================================================
# SAMPLE OUTPUT - Current Status
# =============================================================================
# Show sample of data with all calculated fields

sample_cols = [
    'warehouse_id', 'product_id', 'sku','wac1','wac_p','new_wac',
    # P80/P70 benchmarks and std
    'p80_daily_240d', 'std_daily_240d', 'p70_daily_retailers_240d', 'std_daily_retailers_240d',
    # Current cart rule
    'current_cart_rule',
    # UTH performance (with std thresholds)
    'uth_qty', 'uth_qty_target', 'uth_qty_std', 'uth_qty_status',
    'uth_retailers', 'uth_rets_target', 'uth_rets_std', 'uth_rets_status',
    # Last hour performance (with std thresholds)
    'last_hour_qty', 'last_hour_qty_target', 'last_hour_qty_std', 'last_hour_qty_status',
    'last_hour_retailers', 'last_hour_rets_target', 'last_hour_rets_std', 'last_hour_rets_status'
]

# Filter to columns that exist
sample_cols = [c for c in sample_cols if c in df.columns]

print(f"\n{'='*60}")
print("SAMPLE DATA (First 10 rows with UTH > 0)")
print(f"{'='*60}")
sample = df[df['uth_qty'] > 0][sample_cols].head(10)
display(sample)



SAMPLE DATA (First 10 rows with UTH > 0)


,warehouse_id,product_id,sku,wac1,wac_p,new_wac,p80_daily_240d,std_daily_240d,p70_daily_retailers_240d,std_daily_retailers_240d,...,uth_rets_std,uth_rets_status,last_hour_qty,last_hour_qty_target,last_hour_qty_std,last_hour_qty_status,last_hour_retailers,last_hour_rets_target,last_hour_rets_std,last_hour_rets_status
0,501,19968,مولفيكس بريميوم ماكسى جامبو اقتصادى مقاس4- 56...,326.981375,292.770453,292.770453,5.0,1.035950,2.000,0.479086,...,0.133277,growing,0.0,0.205868,0.042654,dropping,0.0,0.082947,0.019869,dropping
1,236,19968,مولفيكس بريميوم ماكسى جامبو اقتصادى مقاس4- 56...,326.981375,292.770453,292.770453,5.0,1.000854,2.000,0.531837,...,0.181918,on_track,0.0,0.134310,0.026885,dropping,0.0,0.061591,0.016378,dropping
2,962,19968,مولفيكس بريميوم ماكسى جامبو اقتصادى مقاس4- 56...,326.981375,292.770453,292.770453,5.0,1.428834,2.000,0.600464,...,0.211182,on_track,0.0,0.166069,0.047457,dropping,0.0,0.073316,0.022012,dropping
5,8,25254,تمر واحة القصيم شفاف - 700 جم,75.750000,75.750000,75.750000,5.0,0.957427,2.000,0.577350,...,0.118432,growing,0.0,0.078850,0.015099,dropping,0.0,0.023810,0.006873,dropping
8,339,11952,اوكسي يدوي لافندر - 175 جم,140.812729,136.694745,136.694745,10.0,7.454101,5.000,2.682708,...,0.683245,on_track,0.0,0.331424,0.247046,dropping,0.0,0.171742,0.092147,dropping
10,236,3477,جبنة رودس فيتا بيضاء - 250 جم,535.000000,535.000000,535.000000,14.0,6.983517,11.000,6.003962,...,2.150366,on_track,0.0,0.518841,0.258810,dropping,0.0,0.388233,0.211903,dropping
11,962,3477,جبنة رودس فيتا بيضاء - 250 جم,535.000000,535.000000,535.000000,18.0,7.754300,13.500,6.305751,...,2.383253,on_track,0.0,0.618721,0.266541,dropping,0.0,0.468350,0.218763,dropping
14,632,12345,زيووو كرانشى الجبنة - 5 جنية,45.230000,41.614740,41.614740,15.4,6.117286,6.599,3.019232,...,0.848719,growing,0.0,0.744641,0.295791,dropping,0.0,0.364427,0.166736,dropping
18,337,6872,حفاضات مولفيكس حديث الولادة مقاس 1 - 60 حفاضة,277.081992,250.841730,250.849845,6.0,3.890988,2.000,1.496430,...,0.364279,growing,0.0,0.212210,0.137618,dropping,0.0,0.075188,0.056256,dropping
19,8,6872,حفاضات مولفيكس حديث الولادة مقاس 1 - 60 حفاضة,277.081992,250.841730,250.849845,12.0,9.020031,3.000,2.382293,...,0.553730,growing,0.0,0.429400,0.322766,dropping,0.0,0.119162,0.094627,dropping


In [20]:
# =============================================================================
# UPLOAD RESULTS TO SNOWFLAKE AND SEND SLACK NOTIFICATION
# =============================================================================
from common_functions import upload_dataframe_to_snowflake, send_text_slack

# Add created_at as TIMESTAMP (module runs hourly)
df_action['created_at'] = datetime.now(CAIRO_TZ).replace(second=0, microsecond=0)

# Upload to Snowflake
print(f"\n{'='*60}")
print("UPLOADING RESULTS TO SNOWFLAKE")
print(f"{'='*60}")

upload_status = upload_dataframe_to_snowflake(
    "Egypt", 
    df_action, 
    "MATERIALIZED_VIEWS", 
    "pricing_hourly_push", 
    "append", 
    auto_create_table=True, 
    conn=None
)

# Prepare status variables
prices_pushed = push_result.get('pushed', 0) if 'push_result' in dir() else 0
prices_failed = push_result.get('failed', 0) if 'push_result' in dir() else 0
cart_rules_pushed = cart_result.get('pushed', 0) if 'cart_result' in dir() else 0
cart_rules_failed = cart_result.get('failed', 0) if 'cart_result' in dir() else 0

# Count price and cart rule actions
price_changes = len(df_action[df_action['price_action'] != 'none']) if 'price_action' in df_action.columns else 0
cart_changes = len(df_action[df_action['cart_rule_action'] != 'none']) if 'cart_rule_action' in df_action.columns else 0

if upload_status:
    slack_message = f"""✅ *Module 4 - Hourly Updates Completed*

📅 Date: {datetime.now(CAIRO_TZ).strftime('%Y-%m-%d')}
⏰ Hour: {datetime.now(CAIRO_TZ).strftime('%H:%M:%S')} Cairo time
🔧 Mode: {PUSH_MODE.upper()}

📊 *Results:*
• Total SKUs analyzed: {len(df):,}
• SKUs requiring action: {len(df_action):,}
• Price changes: {price_changes:,}
• Cart rule changes: {cart_changes:,}

📤 *Push Status:*
• 💰 Prices: ✅ {prices_pushed} pushed | ❌ {prices_failed} failed
• 🛒 Cart Rules: ✅ {cart_rules_pushed} pushed | ❌ {cart_rules_failed} failed

🗄️ Results uploaded to: MATERIALIZED_VIEWS.pricing_hourly_push"""
    
    send_text_slack('new-pricing-logic', slack_message)
    print("✅ Slack notification sent!")
    print(f"✅ {len(df_action)} records uploaded to Snowflake")
else:
    error_message = f"""❌ *Module 4 - Hourly Updates Failed*

📅 Date: {datetime.now(CAIRO_TZ).strftime('%Y-%m-%d')}
⏰ Hour: {datetime.now(CAIRO_TZ).strftime('%H:%M:%S')} Cairo time
⚠️ Upload to Snowflake failed - please check logs

📤 *Push Status (before upload failure):*
• 💰 Prices: ✅ {prices_pushed} pushed | ❌ {prices_failed} failed
• 🛒 Cart Rules: ✅ {cart_rules_pushed} pushed | ❌ {cart_rules_failed} failed"""
    
    send_text_slack('new-pricing-logic', error_message)
    print("❌ Error notification sent to Slack!")



UPLOADING RESULTS TO SNOWFLAKE
/home/ec2-user/service_account_key.json


/home/ec2-user/SageMaker/Pricing Runs/Prediction_Scripts_2/Happy_hour/git/Mustafa/Pricing Logic/modules/../common_functions.py:698: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, _, _, _ = write_pandas(
/home/ec2-user/anaconda3/envs/python3/lib/python3.12/site-packages/slack/deprecation.py:14: UserWarning: slack package is deprecated. Please use slack_sdk.web/webhook/rtm package instead. For more info, go to https://docs.slack.dev/tools/python-slack-sdk/v3-migration/
  warnings.warn(message)


/home/ec2-user/service_account_key.json
Message Sent
✅ Slack notification sent!
✅ 224 records uploaded to Snowflake
